## _Evaluation Metrics_

_If **GNNBuilder** callback has been run during training, just load data from `dnn_processed/test` and extract `scores` and `y_pid ~ truth` and simply run the following metrics_.

In [ ]:
import sys, os, glob, yaml

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pprint
from tqdm import tqdm
import trackml.dataset

In [ ]:
import torch
import torchmetrics
import pytorch_lightning as pl
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import itertools

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# append parent dir
sys.path.append('..')

### Evaluation Definitions

Metrics to evaluate the GNN networks:

- Accuracy/ACC = $TP+TN/TP+TN+FP+FN$
- sensitivity, recall, hit rate, or true positive rate ($TPR = 1 - FNR$)
- specificity, selectivity or true negative rate ($TNR = 1 - FPR$)
- miss rate or false negative rate ($FNR = 1 - TPR$)
- fall-out or false positive rate ($FPR = 1 - TNR$)
- F1-score = $2 \times (\text{PPV} \times \text{TPR})/(\text{PPV} + \text{TPR})$
- Efficiency/Recall/Sensitivity/Hit Rate: $TPR = TP/(TP+FN)$
- Purity/Precision/Positive Predictive Value: $PPV = TP/(TP+FP$
- AUC-ROC Curve $\equiv$ FPR ($x-$axis) v.s. TPR ($y-$axis) plot
- AUC-PRC Curve $\equiv$ TPR ($x-$axis) v.s. PPV ($y-$axis) plot


Use _`tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()`_ to directly access TN, FP, FN and TP using Scikit-learn.

### Classifier Evaluation

In [ ]:
# fetch all files
# inputdir = "run_all/gnn_processed/test"
# inputdir = "run_all/dnn_processed_bn/test"
# inputdir = "run_all/dnn_processed_ln/test"

# HypGNN (FWP + Filtering)
inputdir = "run_all/fwp_gnn_processed/pred"

# HypGNN (FWP + No Filtering)
# inputdir = "run_all/gnn_processed_fwp_nf/test"

In [ ]:
test_files = sorted(glob.glob(os.path.join(inputdir, "*")))

In [ ]:
# Let's test a event
test = torch.load(test_files[0], map_location=device)

In [ ]:
test

### _Append Scores and Truths_
- _Load all `truth` and `scores` from the `testset` from the `DNN` stage_

In [ ]:
scoresl, truthsl = [], []

for e in range(len(test_files)):
    
    # read test events e.g. gnn_processed/test
    data = torch.load(test_files[e], map_location=device)
    
    # get truths and scores
    truth = data.y_pid
    score = data.scores
    score = score[:truth.size(0)]
    
    # logging
    if e !=0 and (e)%1000==0:
        print("Processed Batches: ", e)
        
    # append each batch
    scoresl.append(score)
    truthsl.append(truth)

In [ ]:
scores = torch.cat(scoresl)
truths = torch.cat(truthsl)

In [ ]:
# save scores and truths as .npy files
# np.save("scores.npy", scores.numpy())
# np.save("truths.npy", truths.numpy())

### _Compute Metrics_

In [ ]:
from src.metric_utils import compute_metrics, plot_metrics
from src.metric_utils import plot_roc, plot_prc, plot_prc_thr, plot_epc, plot_epc_cut, plot_output

In [ ]:
# torch to numpy
scores = scores.numpy()
truths = truths.numpy()

In [ ]:
metrics = compute_metrics(scores,truths,threshold=0.5)

In [ ]:
# Curves
# metrics.prc_precision, metrics.prc_recall, metrics.prc_thresh
# metrics.roc_tpr, metrics.roc_fpr, metrics.roc_thresh

In [ ]:
metrics.accuracy

In [ ]:
metrics.recall

In [ ]:
metrics.precision

In [ ]:
metrics.f1

In [ ]:
metrics.prc_precision.shape

### _(a) - Plot Metrics_

In [ ]:
outname = "fwp_wf"

In [ ]:
plot_metrics(scores,truths, metrics, name=outname)

In [ ]:
# ROC Curve
plot_roc(metrics, name=outname)

In [ ]:
# PR Curve
plot_prc(metrics, name=outname)

In [ ]:
# Built from PRC Curve
plot_prc_thr(metrics, name=outname)

In [ ]:
# EP Curve from ROC
plot_epc(metrics, name=outname)

In [ ]:
# Built from ROC Curve
plot_epc_cut(metrics, name=outname)

In [ ]:
# Model output: True and False
plot_output(scores, truths, threshold=0.5, name=outname)

### _(b) - S/B Suppression_

Background rejection rate (1/FPR) is given as $1/\epsilon_{bkg}$ where $\epsilon_{bkg}$ is the fraction of fake edges that pass the classification requirement. Signal efficiency (TPR ~ Recall) ($\epsilon_{sig}$) is defined as the number of true edges above a given classification score cut over the total number of true edges. What we have?

- Signal Efficiency = $\epsilon_{sig}$ = TPR ~ Recall 
- Background Rejection = $1 - \epsilon_{bkg}$ ???
- Background Rejection Rate = $1/\epsilon_{bkg}$ = 1/FPR


First apply a edge score cut to binarized the `scores`, we will call it `preds`. The count number of false or true edges that pass this cut. Then calculated background rejection rate and signal efficiency. For making a plot one can do calculations in batch by batch mode on the test dataset.

In [ ]:
sig = metrics.roc_tpr

In [ ]:
bkg_rejection = 1/metrics.roc_fpr

In [ ]:
# cut off eff < 0.2 or 0.5
sig_mask = sig > 0.3

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
ax.plot(sig[sig_mask], bkg_rejection[sig_mask], label="Interaction GNN", color="blue")

# Axes Params
ax.set_xlabel("Signal Efficiency", fontsize=16)
ax.set_ylabel("Background Rejection Rate", fontsize=16)
ax.set_yscale('log')
ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)
ax.grid(True)
ax.legend(fontsize=14, loc='upper right')
    
# Figure Params
fig.tight_layout()
fig.savefig(outname+"_SB.pdf")